This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

In [1]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 2s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.31703615, -0.15522641,  0.37555566, -0.16700734, -0.07459237,
        -0.4965024 , -0.26481807, -0.48051572,  0.3638081 ,  0.08519404]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.07800337, 0.0917036 , 0.15592012, 0.09062959, 0.09940434,
        0.06518868, 0.08218478, 0.0662392 , 0.15409915, 0.11662716]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.7304695

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 887us/step - loss: 0.4860 - accuracy: 0.8566
Epoch 2/5
1875/1875 [==============================] - 2s 895us/step - loss: 0.1504 - accuracy: 0.9553
Epoch 3/5
1875/1875 [==============================] - 2s 900us/step - loss: 0.1094 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 2s 893us/step - loss: 0.0882 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 2s 906us/step - loss: 0.0706 - accuracy: 0.9768


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0761 - accuracy: 0.9763


[0.07614302635192871, 0.9763000011444092]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.0468123e-09, 8.4473872e-10, 2.3839864e-06, 1.9787347e-04,
        1.5870617e-11, 3.5296748e-07, 3.4736360e-15, 9.9978334e-01,
        3.3413205e-08, 1.6038339e-05],
       [7.3820955e-10, 6.5055727e-05, 9.9991715e-01, 1.7748564e-05,
        2.0778226e-15, 5.0203013e-08, 5.1931237e-09, 3.3859037e-15,
        2.9587649e-10, 9.3930702e-15],
       [1.2094479e-08, 9.9900645e-01, 3.1521678e-05, 6.4535552e-06,
        3.8582180e-05, 1.0406454e-06, 1.2546770e-05, 7.7269482e-04,
        1.3035751e-04, 3.5886043e-07],
       [9.9933439e-01, 9.6993418e-08, 6.0354532e-06, 1.1925691e-06,
        7.2719095e-06, 1.4542431e-06, 6.1370118e-04, 3.5587343e-06,
        8.2667245e-10, 3.2186057e-05],
       [1.0158657e-06, 8.2586411e-08, 2.2254644e-06, 2.9496630e-07,
        9.9542612e-01, 1.0464098e-06, 1.2295540e-06, 6.5653649e-04,
        7.6278587e-07, 3.9108139e-03]], dtype=float32)>